# import

In [36]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [37]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [38]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [39]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits, y_true = labels,
                                                                   from_logits = True))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [40]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class version)

In [41]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(4):
            self.model.add(dense(512,weight_init))
            self.model.add(relu())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [42]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(4):
        model.add(dense(512,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [43]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [44]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [45]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.8980, train_loss: 2.04543924, train_accuracy: 0.4609, test_accuracy: 0.3132
Epoch: [ 0] [    1/  468] time: 1.3108, train_loss: 1.97688460, train_accuracy: 0.5078, test_accuracy: 0.4463
Epoch: [ 0] [    2/  468] time: 1.7277, train_loss: 1.77500701, train_accuracy: 0.6875, test_accuracy: 0.5977
Epoch: [ 0] [    3/  468] time: 2.0878, train_loss: 1.63330150, train_accuracy: 0.6562, test_accuracy: 0.6256
Epoch: [ 0] [    4/  468] time: 2.4608, train_loss: 1.26743519, train_accuracy: 0.7266, test_accuracy: 0.6904
Epoch: [ 0] [    5/  468] time: 2.8238, train_loss: 0.99867105, train_accuracy: 0.7969, test_accuracy: 0.6984
Epoch: [ 0] [    6/  468] time: 3.1888, train_loss: 0.82300854, train_accuracy: 0.7812, test_accuracy: 0.6960
Epoch: [ 0] [    7/  468] time: 3.5845, train_loss: 0.70415658, train_accuracy: 0.7891, test_accuracy: 0.7777
Epoch: [ 0] [    8/  468] time: 3.9475, 

Epoch: [ 0] [   74/  468] time: 28.2898, train_loss: 0.20669748, train_accuracy: 0.9375, test_accuracy: 0.9211
Epoch: [ 0] [   75/  468] time: 28.6508, train_loss: 0.27438816, train_accuracy: 0.8984, test_accuracy: 0.9182
Epoch: [ 0] [   76/  468] time: 29.0095, train_loss: 0.17188886, train_accuracy: 0.9453, test_accuracy: 0.9223
Epoch: [ 0] [   77/  468] time: 29.3766, train_loss: 0.24224511, train_accuracy: 0.9375, test_accuracy: 0.9287
Epoch: [ 0] [   78/  468] time: 29.7117, train_loss: 0.16040525, train_accuracy: 0.9609, test_accuracy: 0.9275
Epoch: [ 0] [   79/  468] time: 30.0605, train_loss: 0.20219946, train_accuracy: 0.9297, test_accuracy: 0.9240
Epoch: [ 0] [   80/  468] time: 30.4355, train_loss: 0.25777900, train_accuracy: 0.9141, test_accuracy: 0.9220
Epoch: [ 0] [   81/  468] time: 30.8155, train_loss: 0.18706244, train_accuracy: 0.9688, test_accuracy: 0.9172
Epoch: [ 0] [   82/  468] time: 31.2144, train_loss: 0.24157235, train_accuracy: 0.9375, test_accuracy: 0.9144
E

Epoch: [ 0] [  148/  468] time: 54.7984, train_loss: 0.14816058, train_accuracy: 0.9531, test_accuracy: 0.9400
Epoch: [ 0] [  149/  468] time: 55.1523, train_loss: 0.16601744, train_accuracy: 0.9609, test_accuracy: 0.9429
Epoch: [ 0] [  150/  468] time: 55.5277, train_loss: 0.16494009, train_accuracy: 0.9297, test_accuracy: 0.9453
Epoch: [ 0] [  151/  468] time: 55.8907, train_loss: 0.09567340, train_accuracy: 0.9688, test_accuracy: 0.9438
Epoch: [ 0] [  152/  468] time: 56.2499, train_loss: 0.16247128, train_accuracy: 0.9531, test_accuracy: 0.9426
Epoch: [ 0] [  153/  468] time: 56.6368, train_loss: 0.10699805, train_accuracy: 0.9688, test_accuracy: 0.9386
Epoch: [ 0] [  154/  468] time: 56.9909, train_loss: 0.16512731, train_accuracy: 0.9688, test_accuracy: 0.9363
Epoch: [ 0] [  155/  468] time: 57.3330, train_loss: 0.13863555, train_accuracy: 0.9609, test_accuracy: 0.9374
Epoch: [ 0] [  156/  468] time: 57.6811, train_loss: 0.20286635, train_accuracy: 0.9453, test_accuracy: 0.9419
E

Epoch: [ 0] [  222/  468] time: 81.3290, train_loss: 0.17545332, train_accuracy: 0.9531, test_accuracy: 0.9527
Epoch: [ 0] [  223/  468] time: 81.7040, train_loss: 0.13420400, train_accuracy: 0.9688, test_accuracy: 0.9549
Epoch: [ 0] [  224/  468] time: 82.0570, train_loss: 0.11419959, train_accuracy: 0.9688, test_accuracy: 0.9577
Epoch: [ 0] [  225/  468] time: 82.3931, train_loss: 0.05782131, train_accuracy: 0.9922, test_accuracy: 0.9600
Epoch: [ 0] [  226/  468] time: 82.7502, train_loss: 0.09383298, train_accuracy: 0.9844, test_accuracy: 0.9609
Epoch: [ 0] [  227/  468] time: 83.0887, train_loss: 0.11244226, train_accuracy: 0.9688, test_accuracy: 0.9600
Epoch: [ 0] [  228/  468] time: 83.4537, train_loss: 0.18171379, train_accuracy: 0.9375, test_accuracy: 0.9584
Epoch: [ 0] [  229/  468] time: 83.8098, train_loss: 0.10126878, train_accuracy: 0.9766, test_accuracy: 0.9570
Epoch: [ 0] [  230/  468] time: 84.1676, train_loss: 0.16898572, train_accuracy: 0.9297, test_accuracy: 0.9556
E

Epoch: [ 0] [  296/  468] time: 108.9339, train_loss: 0.18650106, train_accuracy: 0.9609, test_accuracy: 0.9536
Epoch: [ 0] [  297/  468] time: 109.3797, train_loss: 0.22506037, train_accuracy: 0.9531, test_accuracy: 0.9537
Epoch: [ 0] [  298/  468] time: 109.9033, train_loss: 0.11883865, train_accuracy: 0.9531, test_accuracy: 0.9564
Epoch: [ 0] [  299/  468] time: 110.4133, train_loss: 0.09387672, train_accuracy: 0.9688, test_accuracy: 0.9577
Epoch: [ 0] [  300/  468] time: 110.8577, train_loss: 0.09249623, train_accuracy: 0.9766, test_accuracy: 0.9610
Epoch: [ 0] [  301/  468] time: 111.2646, train_loss: 0.11938502, train_accuracy: 0.9766, test_accuracy: 0.9611
Epoch: [ 0] [  302/  468] time: 111.7304, train_loss: 0.10178491, train_accuracy: 0.9688, test_accuracy: 0.9603
Epoch: [ 0] [  303/  468] time: 112.1572, train_loss: 0.10480646, train_accuracy: 0.9766, test_accuracy: 0.9545
Epoch: [ 0] [  304/  468] time: 112.6230, train_loss: 0.15373108, train_accuracy: 0.9531, test_accuracy:

Epoch: [ 0] [  370/  468] time: 141.1842, train_loss: 0.06947529, train_accuracy: 0.9922, test_accuracy: 0.9517
Epoch: [ 0] [  371/  468] time: 141.5512, train_loss: 0.16872644, train_accuracy: 0.9453, test_accuracy: 0.9541
Epoch: [ 0] [  372/  468] time: 141.9661, train_loss: 0.07984887, train_accuracy: 0.9844, test_accuracy: 0.9578
Epoch: [ 0] [  373/  468] time: 142.3331, train_loss: 0.11256307, train_accuracy: 0.9688, test_accuracy: 0.9603
Epoch: [ 0] [  374/  468] time: 142.7340, train_loss: 0.05986298, train_accuracy: 0.9844, test_accuracy: 0.9616
Epoch: [ 0] [  375/  468] time: 143.1020, train_loss: 0.24038211, train_accuracy: 0.9453, test_accuracy: 0.9630
Epoch: [ 0] [  376/  468] time: 143.4890, train_loss: 0.15537539, train_accuracy: 0.9453, test_accuracy: 0.9640
Epoch: [ 0] [  377/  468] time: 143.9388, train_loss: 0.11842951, train_accuracy: 0.9688, test_accuracy: 0.9630
Epoch: [ 0] [  378/  468] time: 144.2938, train_loss: 0.10617097, train_accuracy: 0.9688, test_accuracy:

Epoch: [ 0] [  444/  468] time: 171.7836, train_loss: 0.10463088, train_accuracy: 0.9609, test_accuracy: 0.9688
Epoch: [ 0] [  445/  468] time: 172.1486, train_loss: 0.08639856, train_accuracy: 0.9766, test_accuracy: 0.9684
Epoch: [ 0] [  446/  468] time: 172.5398, train_loss: 0.09968309, train_accuracy: 0.9844, test_accuracy: 0.9680
Epoch: [ 0] [  447/  468] time: 172.9185, train_loss: 0.08262815, train_accuracy: 0.9688, test_accuracy: 0.9678
Epoch: [ 0] [  448/  468] time: 173.2746, train_loss: 0.09664238, train_accuracy: 0.9922, test_accuracy: 0.9680
Epoch: [ 0] [  449/  468] time: 173.6316, train_loss: 0.09677390, train_accuracy: 0.9766, test_accuracy: 0.9666
Epoch: [ 0] [  450/  468] time: 174.0091, train_loss: 0.10005379, train_accuracy: 0.9766, test_accuracy: 0.9667
Epoch: [ 0] [  451/  468] time: 174.3970, train_loss: 0.06209824, train_accuracy: 0.9922, test_accuracy: 0.9677
Epoch: [ 0] [  452/  468] time: 174.7820, train_loss: 0.06860485, train_accuracy: 0.9844, test_accuracy:

In [46]:
# test acc = 96.28